In this notebook, my goal is to select the top N articles with the highest proportion of each distinct topic. This allows me to explore the key discussions associated with each topic.

First, I load the datasets from Handelsblatt, SZ, Welt, and dpa.

In [1]:
import os
import pandas as pd
from ast import literal_eval
from datetime import datetime
startTime = datetime.now()

# Set the path variable to point to the 'newspaper_data_processing' directory
path = os.getcwd().replace('\\nowcasting_with_text\\topics', '\\newspaper_data_processing')

# Load pre-processed 'dpa' dataset from a CSV file
dpa = pd.read_csv(path + '\\dpa\\' + 'dpa_prepro_final.csv', encoding = 'utf-8', sep=';', index_col = 0,  keep_default_na=False,
                   dtype = {'rubrics': 'str', 
                            'source': 'str',
                            'keywords': 'str',
                            'title': 'str',
                            'city': 'str',
                            'genre': 'str',
                            'wordcount': 'str'},
                  converters = {'paragraphs': literal_eval})

# Keep only the article texts and their respective publication dates
dpa = dpa[['texts', 'day', 'month', 'year']]

# Load pre-processed 'SZ' dataset from a CSV file
sz = pd.read_csv(path + '\\SZ\\' + 'sz_prepro_final.csv', encoding = 'utf-8-sig', sep=';', index_col = 0, dtype = {'newspaper': 'str',
                                                                                                 'newspaper_2': 'str',
                                                                                                 'quelle_texts': 'str',
                                                                                                 'page': 'str',
                                                                                                 'rubrics': 'str'})
sz.page = sz.page.fillna('')
sz.newspaper = sz.newspaper.fillna('')
sz.newspaper_2 = sz.newspaper_2.fillna('')
sz.rubrics = sz.rubrics.fillna('')
sz.quelle_texts = sz.quelle_texts.fillna('')

# Keep only the article texts and their respective publication dates
sz = sz[['texts', 'day', 'month', 'year']]

# Load pre-processed 'Handelsblatt' dataset from a CSV file
hb = pd.read_csv(path + '\\Handelsblatt\\' + 'hb_prepro_final.csv', encoding = 'utf-8-sig', sep=';', index_col = 0, dtype = {'kicker': 'str',
                                                                                                 'page': 'str',
                                                                                                 'series_title': 'str',
                                                                                                 'rubrics': 'str'})
hb.page = hb.page.fillna('')
hb.series_title = hb.series_title.fillna('')
hb.kicker = hb.kicker.fillna('')
hb.rubrics = hb.rubrics.fillna('')

# Keep only the article texts and their respective publication dates
hb = hb[['texts', 'day', 'month', 'year']]

# Load pre-processed 'Welt' dataset from a CSV file
welt = pd.read_csv(path + '\\Welt\\' + 'welt_prepro_final.csv', encoding = 'utf-8-sig', sep=';', index_col = 0, dtype = {'newspaper': 'str',
                                                                                                 'rubrics': 'str',
                                                                                                 'title': 'str'})
welt.title = welt.title.fillna('')
welt.rubrics = welt.rubrics.fillna('')

# Keep only the article texts and their respective publication dates
welt = welt[['texts', 'day', 'month', 'year']]

# Concatenate the 'dpa', 'sz', 'hb', and 'welt' DataFrames into a single DataFrame 'data'
data = pd.concat([dpa, sz, hb, welt])

# The number of articles in the final dataset
print(len(data))

# Sort the data in chronological order
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True])
# Reset the index of the DataFrame
data.reset_index(inplace=True, drop=True)
data.head()

3336299


,texts,day,month,year
0,Schalck: Milliardenkredit sicherte Zahlungsfäh...,1,1,1991
1,Welajati: Iran bleibt bei einem Krieg am Golf ...,1,1,1991
2,Bush will offenbar seinen Außenminister erneut...,1,1,1991
3,Sperrfrist 1. Januar 1000 HBV fordert umfassen...,1,1,1991
4,Schamir weist Nahost-Äußerungen des neuen EG-P...,1,1,1991


Afterward, I am incorporating the topic distributions for each article, which were previously computed using the Latent Dirichlet Allocation (LDA) algorithm in the notebook named `Topic model estimation`. 

In [2]:
# Load the article topics from a CSV file
article_topics = pd.read_csv('article_topic.csv', encoding='utf-8', index_col=0)

# Merge the `data` DataFrame with the `article_topics` DataFrame
data = pd.concat([data, article_topics], axis=1)

Now I identify the top `n` articles with the highest proportion of a certain topic, and for each of these articles, I print the date of publication, the text, and the proportion of this topic in the article.

In [3]:
# Set the number of articles
n = 1

# Set the topic of interest
topic = 'T50'

# Sort the dataframe by the topic in descending order and pick the top n articles
top_articles = data.sort_values(by=[topic], ascending=False).head(n)

# Print the texts of the top articles and their topic proportion
for i, row in top_articles.iterrows():
    print(f'Article {i}:')
    print('Date of Publication:', f"{row['day']}-{row['month']}-{row['year']}")
    print('Text:', row['texts'])
    print('Topic proportion:', row[topic])
    print('\n---\n')

Article 2016094:
Date of Publication: 11-8-2007
Text: Die Angst vor einem Crash geht um. Notenbanken geben wieder Geld ins Finanzsystem - Aktienkurse fallen - Furcht vor Folgen für Wirtschaft. Frankfurt/Berlin - Aus Angst vor einer Kreditkrise mit Auswirkungen für die Gesamtwirtschaft haben Zentralbanken am Freitag erneut Milliardensummen in das internationale Bankensystem gepumpt. Die EZB stellte den Kreditinstituten noch einmal gut 61 Mrd. Euro zu einem Zinssatz von 4,08 Prozent zusätzlich zur Verfügung, die amerikanische Fed 19 Mrd. Dollar. Damit wurden die Notenbanker zum zweiten Mal innerhalb von 24 Stunden aktiv. Die Währungshüter wollen mit der Maßnahme die Angst an den weltweiten Kapitalmärkten dämpfen. Sie fürchten, dass die Banken ansonsten die Geldvergabe weitgehend einstellen könnten. Die Folge wäre eine Kreditklemme, unter der am Ende auch die Volkswirtschaften leiden würden. Den Anlegern aber scheinen die Liquiditätsspritzen der Notenbanker eher Angst einzujagen, dass die

I also identify articles from a specific year with the highest share of a selected topic to understand the key events covered at that time within the context of that topic.

In [4]:
# Set the number of articles 
n = 1

# Set the topic of interest
topic = 'T29'

# Filter articles from the year 2008
filtered_data = data[data['year']==2008]

# Sort the filtered dataframe by the topic in descending order and pick the top n articles
top_articles = filtered_data.sort_values(by=[topic], ascending=False).head(n)

# Print the texts of the top articles and their topic proportion
for i, row in top_articles.iterrows():
    print(f'Article {i + 1}:')
    print('Date of Publication:', f"{row['day']}-{row['month']}-{row['year']}")
    print('Text:', row['texts'])
    print('Topic proportion:', row[topic])
    print('\n---\n')

Article 2089309:
Date of Publication: 26-2-2008
Text: Dresdner Bank greift mit Billigmarke an. Frankfurt/Main - Die Dresdner Bank will in den nächsten Monaten mit einer Billigmarke auf den Markt kommen. Unter dem Namen "Dresdner Direct 24" sollen Kunden Standardprodukte über Internet und in den Filialen erhalten. Neben der Premium-Marke Dresdner Bank werde auf diese Weise ein Discount-Angebot geschaffen, sagte Vorstandschef Herbert Walter. Der Start ist noch für das zweite Quartal geplant. Dem Vernehmen nach wird das Angebot vom Girokonto über Aktiengeschäfte und Festgeld bis hin zur Baufinanzierung reichen. Es richtet sich vor allem an Kunden, die auf Beratung keinen großen Wert legen. Dresdner-Bank-Chef Walter ist sich dabei der Gefahr bewusst, dass sich beide Marken gegenseitig Kunden streitig machen. "Das ist immer noch besser, als wenn die Kunden zu einer anderen Bank wechseln würden." Die Deutsche Bank hat sich mit der Übernahme der Norisbank ebenfalls eine Billigmarke zugelegt. 